In [124]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ghsperson-house/Person files/Person files/General Household Survey 2015 (House file).pdf
/kaggle/input/ghsperson-house/Person files/Person files/AJAD_2011_9_2_5Pede.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON.pdf
/kaggle/input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018_F1.csv
/kaggle/input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018.pdf
/kaggle/input/ghsperson-house/Person 

In [125]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.utils import resample
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import metrics

In [126]:
#import the household file data
house2011 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2011-HOUSE_F1.csv')
house2012 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2012-HOUSE_F1.csv')
house2013 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2013-HOUSE_F1.csv')
house2014 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2014-HOUSE_F1.csv')
house2015 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2015-HOUSE_F1.csv')
house2016 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2016-HOUSE_F1.csv')
house2017 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2017-HOUSE_F1.csv')
house2018 = pd.read_csv('../input/ghsperson-house/House files/House files/GHS-2018-HOUSE_F1.csv')

In [127]:
#import the person file data
person2011 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-Person-2011/GHS-Person-2011_F1.csv')
person2012 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2012-PERSON/GHS-2012-PERSON_F1.csv')
person2013 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2013-PERSON (1)/GHS-2013-PERSON_F1.csv')
person2014 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-Person-2014-rwt2018/GHS-Person-2014-rwt2018_F1.csv')
person2015 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2015-PERSON/GHS-2015-PERSON_F1.csv')
person2016 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2016-PERSONs/GHS-2016-PERSONs_F1.csv')
person2017 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2017-PERSON/GHS-2017-PERSON_F1.csv')
person2018 = pd.read_csv('../input/ghsperson-house/Person files/Person files/GHS-2018-PERSON/GHS-2018-PERSON_F1.csv')

In [128]:
def rename_columns(df):
    return df.rename(columns = {'UqNr': 'uqnr', 'PersonNR': 'personnr','Personnr': 'personnr', 'Q16hiedu':'Q15HIEDU','Q11relsh':'Q11RELSH','q11RELSH':'Q11RELSH'})

In [129]:
def get_renamed_dfs(dfs):
    new_dfs = []
    for df in dfs:
        new_df = rename_columns(df)
        new_dfs.append(new_df)
    return new_dfs

In [130]:
house_dfs = [house2011, house2012, house2013, house2014, house2015, house2016, house2017, house2018]
renamed_house_dfs = get_renamed_dfs(house_dfs)
house_file_data = pd.concat(renamed_house_dfs)
print(house_file_data.columns)

Index(['uqnr', 'personnr', 'PSU', 'Prov', 'head_popgrp', 'head_sex',
       'head_age', 'Q31MainD', 'Q31OthrD', 'Q32roof',
       ...
       'Q82aNAgri', 'Q82bNpersno', 'Q83grai', 'Q83fruit', 'Q87Chickens',
       'Q88dType', 'Q815bTotveh', 'Q528cFBE', 'Q66bfixedint', 'Q66bmobint'],
      dtype='object', length=977)


In [131]:
person_dfs = [person2011, person2012, person2013, person2014, person2015, person2016, person2017, person2018]
renamed_person_dfs = get_renamed_dfs(person_dfs)
person_file_data = pd.concat(renamed_person_dfs )
print(person_file_data.columns)

Index(['uqnr', 'personnr', 'PSU', 'Prov', 'B_Nights', 'Gender', 'Age',
       'Age_grp', 'Race', 'Q11RELSH',
       ...
       'Q18STORIES', 'Q18SING', 'Q18DRAW', 'Q18TALK', 'Q18COUNT', 'Q18NAME',
       'Q18PLAY', 'Q23MAL', 'Q26MAL', 'Prov_add_1588231763189'],
      dtype='object', length=499)


In [132]:
# house_file_data.to_csv('house_file_data.csv')
# person_file_data.to_csv('person_file_data.csv')

In [133]:
# person_file_data = person_file_data.iloc[0:10000]
# house_file_data = house_file_data.iloc[0:10000]

In [134]:
print('person_rows: {:}'.format(person_file_data.shape[0]))
print('person_columns: {:}'.format(person_file_data.shape[1]))
print("\n")
print('house_rows: {:}'.format(house_file_data.shape[0]))
print('house_columns: {:}'.format(house_file_data.shape[1]))

person_rows: 661980
person_columns: 499


house_rows: 186517
house_columns: 977


In [135]:
house_data_chosen_columns = ['uqnr', 'personnr','totmhinc', 'hholdsz', 'head_age', 'head_sex']
person_data_chosen_columns = ['uqnr', 'personnr','employ_Status1','employ_Status2','Q15HIEDU', 'Q11RELSH']
# 'Q812NetIncome'

In [136]:
house_data_chosen_subset = house_file_data[house_data_chosen_columns]
house_data_chosen_subset = house_data_chosen_subset.dropna()
house_data_chosen_subset['personnr'].head()

0    1
1    1
2    1
3    1
4    1
Name: personnr, dtype: int64

In [137]:
person_data_chosen_subset = person_file_data[person_data_chosen_columns]
person_data_chosen_subset = person_data_chosen_subset.dropna()
person_data_chosen_subset.head()

,uqnr,personnr,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
0,101000170000001501,1,1.0,1.0,12.0,1
1,101000170000001501,2,1.0,1.0,23.0,2
2,101000170000001501,3,8.0,8.0,98.0,3
3,101000170000005701,1,1.0,1.0,10.0,1
4,101000170000005701,2,9.0,3.0,1.0,2


In [138]:
# left = house_data_chosen_subset.set_index(['uqnr','personnr'])
# right = person_data_chosen_subset.set_index(['uqnr','personnr'])

# joined_data0 = left.join(right, lsuffix='_house', rsuffix='_person')
# joined_data0.head(50)

In [139]:
left = house_data_chosen_subset.set_index('uqnr')
right = person_data_chosen_subset.set_index('uqnr')

joined_data0 = left.join(right, lsuffix='_house', rsuffix='_person')
joined_data0.head()

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,2,31,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,10000,2,31,1,3.0,8.0,8.0,98.0,3.0
101000170000001501,1,10000,3,32,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,3,32,1,2.0,1.0,1.0,23.0,2.0


In [140]:
joined_data0.isnull().sum()

personnr_house         0
totmhinc               0
hholdsz                0
head_age               0
head_sex               0
personnr_person    31898
employ_Status1     31898
employ_Status2     31898
Q15HIEDU           31898
Q11RELSH           31898
dtype: int64

In [141]:
joined_data0.isnull().sum()['employ_Status1'] / joined_data0.shape[0]

0.023655448287727615

In [142]:
#Temporarily reducing the data for cleaning purposes
joined_data = joined_data0.iloc[0:1000]
joined_data.head()

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,2,31,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,10000,2,31,1,3.0,8.0,8.0,98.0,3.0
101000170000001501,1,10000,3,32,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,3,32,1,2.0,1.0,1.0,23.0,2.0


In [143]:
final_joined_data0 = joined_data.drop_duplicates()
final_joined_data0.head() 

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,2,31,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,10000,2,31,1,3.0,8.0,8.0,98.0,3.0
101000170000001501,1,10000,3,32,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,3,32,1,2.0,1.0,1.0,23.0,2.0


In [144]:
#Drop missing columns if they are less than 30% of the data 
final_joined_data10 = final_joined_data0.dropna()
final_joined_data10.head(30)

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,employ_Status1,employ_Status2,Q15HIEDU,Q11RELSH
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,2,31,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,10000,2,31,1,3.0,8.0,8.0,98.0,3.0
101000170000001501,1,10000,3,32,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,10000,3,32,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,10000,3,32,1,3.0,8.0,8.0,98.0,3.0
101000170000001501,1,13000,3,33,1,1.0,1.0,1.0,12.0,1.0
101000170000001501,1,13000,3,33,1,2.0,1.0,1.0,23.0,2.0
101000170000001501,1,13000,3,33,1,3.0,8.0,8.0,98.0,3.0


In [145]:
def create_empl_status_col(df):
    empl_statuses = []
    df['employ_status'] = [i for i in range(len(df))]
    for i in range(len(df)):
        empl_status = None
        relation_to_head = df.iloc[i]['Q11RELSH']
        empl_status1 = df.iloc[i]['employ_Status1']
        empl_status2 = df.iloc[i]['employ_Status2']
        if empl_status1 == 1.0 or empl_status2 == 1.0:
            empl_status = 1
            empl_statuses.append(empl_status)
        else:
            empl_status = 0
            empl_statuses.append(empl_status)
    df['employ_status'] = empl_statuses
    return df

In [146]:
final_joined_data1 = create_empl_status_col(final_joined_data10)
final_joined_data2 = final_joined_data1.drop(['employ_Status1', 'employ_Status2'], axis = 1)
final_joined_data2.head(10)

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,Q15HIEDU,Q11RELSH,employ_status
uqnr,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,12.0,1.0,1
101000170000001501,1,10000,2,31,1,2.0,23.0,2.0,1
101000170000001501,1,10000,2,31,1,3.0,98.0,3.0,0
101000170000001501,1,10000,3,32,1,1.0,12.0,1.0,1
101000170000001501,1,10000,3,32,1,2.0,23.0,2.0,1
101000170000001501,1,10000,3,32,1,3.0,98.0,3.0,0
101000170000001501,1,13000,3,33,1,1.0,12.0,1.0,1
101000170000001501,1,13000,3,33,1,2.0,23.0,2.0,1
101000170000001501,1,13000,3,33,1,3.0,98.0,3.0,0


In [147]:
final_joined_data2.isnull().sum()

personnr_house     0
totmhinc           0
hholdsz            0
head_age           0
head_sex           0
personnr_person    0
Q15HIEDU           0
Q11RELSH           0
employ_status      0
dtype: int64

In [148]:
#Drop missing columns if they are less than 30% of the data 
final_joined_data3 = final_joined_data2.dropna()
final_joined_data3.head(10)

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,Q15HIEDU,Q11RELSH,employ_status
uqnr,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,12.0,1.0,1
101000170000001501,1,10000,2,31,1,2.0,23.0,2.0,1
101000170000001501,1,10000,2,31,1,3.0,98.0,3.0,0
101000170000001501,1,10000,3,32,1,1.0,12.0,1.0,1
101000170000001501,1,10000,3,32,1,2.0,23.0,2.0,1
101000170000001501,1,10000,3,32,1,3.0,98.0,3.0,0
101000170000001501,1,13000,3,33,1,1.0,12.0,1.0,1
101000170000001501,1,13000,3,33,1,2.0,23.0,2.0,1
101000170000001501,1,13000,3,33,1,3.0,98.0,3.0,0


In [149]:
final_joined_data2.index[0]

101000170000001501

In [150]:
head_empl_infos = []
for i in range(len(final_joined_data3)):
    relation_to_head = final_joined_data3.iloc[i]['Q11RELSH']
    if relation_to_head == 1:
        house_id = final_joined_data3.index[i]
        head_empl_status = final_joined_data3.iloc[i]['employ_status']
        head_empl_info = {house_id : head_empl_status}
        head_empl_infos.append(head_empl_info)
        


In [151]:
# head_empl_infos = []
head_employ_statuses = [0 for i in range(len(final_joined_data3))]
final_joined_data3['head_employ_status'] = [0 for i in range(len(final_joined_data3))]
for i in range(len(final_joined_data3)):
    house_id = final_joined_data3.index[i]
    for dict_entry in head_empl_infos:
        for key in dict_entry:
            if key == house_id:
                head_employ_statuses[i] = dict_entry[key]
    
final_joined_data3['head_employ_status'] = head_employ_statuses   
final_joined_data3.head(30)   
#     for j in range(len(head_empl_infos)):


,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,Q15HIEDU,Q11RELSH,employ_status,head_employ_status
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,2,31,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,10000,2,31,1,3.0,98.0,3.0,0,1.0
101000170000001501,1,10000,3,32,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,3,32,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,10000,3,32,1,3.0,98.0,3.0,0,1.0
101000170000001501,1,13000,3,33,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,13000,3,33,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,13000,3,33,1,3.0,98.0,3.0,0,1.0


In [152]:
final_joined_data11 = final_joined_data3.drop_duplicates()
final_joined_data11.head(10) 

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,Q15HIEDU,Q11RELSH,employ_status,head_employ_status
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,2,31,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,10000,2,31,1,3.0,98.0,3.0,0,1.0
101000170000001501,1,10000,3,32,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,3,32,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,10000,3,32,1,3.0,98.0,3.0,0,1.0
101000170000001501,1,13000,3,33,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,13000,3,33,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,13000,3,33,1,3.0,98.0,3.0,0,1.0


In [153]:
# final_joined_data2['head_employ_status'] = [0 for i in range(len(final_joined_data2))]
# head_employ_status = []
# final_joined_data2.head()

In [159]:
data = final_joined_data11.groupby('uqnr')
# data.drop_duplicates()
data.head()

,personnr_house,totmhinc,hholdsz,head_age,head_sex,personnr_person,Q15HIEDU,Q11RELSH,employ_status,head_employ_status
uqnr,,,,,,,,,,
101000170000001501,1,10000,2,31,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,2,31,1,2.0,23.0,2.0,1,1.0
101000170000001501,1,10000,2,31,1,3.0,98.0,3.0,0,1.0
101000170000001501,1,10000,3,32,1,1.0,12.0,1.0,1,1.0
101000170000001501,1,10000,3,32,1,2.0,23.0,2.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...
104000630000001901,1,4500,3,78,1,2.0,11.0,2.0,0,1.0
104000630000001901,1,4500,3,78,1,3.0,9.0,3.0,0,1.0
104000630000003801,1,3900,2,85,2,1.0,10.0,1.0,1,1.0


In [155]:
# for i, group in data:
#     print(group['employ_Status1'])
# #     if final_joined_data['employ_Status1'][i] == 1.0:
# #         print("employed")

In [156]:
# def create_empl_status_col(df):
#     empl_statuses = []
#     empl_status = None
#     empl_status1 = df['employ_Status1']
#     empl_status2 = df['employ_Status2']
#     relation_to_head = df['Q11RELSH']
#     print(len(empl_status1))
#     print(len(empl_status1))
#     print(len(relation_to_head))
#     assert len(empl_status1) == len(empl_status1)
#     assert len(empl_status1) == len(relation_to_head)
    
#     i = 0
    
#     for value in relation_to_head:
#         if value == 1:
#             print(empl_status1[1])
#             i += 1
# #             print(len(empl_status1[i]))
# #             i = i + 1
# #             if empl_status1[i] == 1 or empl_status2[i] == 1:
# #                 empl_status = 1
# #                 empl_statuses.append(empl_status)
# #             else:
# #                 empl_status = 0
# #                 empl_statuses.append(empl_status)
# # #         
# #     df['status'] = empl_statuses
#     return df
# create_empl_status_col(final_joined_data)

In [157]:
create_empl_status_col(final_joined_data)

NameError: name 'final_joined_data' is not defined

In [ ]:
data = create_empl_status_col(final_joined_data)
data.head()

In [ ]:
# final_joined_data = joined_data.drop_duplicates(inplace=True)
# final_joined_data.head()

In [ ]:
print('final_joined_data_rows: {:}'.format(final_joined_data.shape[0]))
print('final_joined_data_columns: {:}'.format(final_joined_data.shape[1]))


In [ ]:
# final_house_data.isnull().sum()
final_joined_data.to_csv('final_joined_data.csv')

In [ ]:
sample = final_joined_data.groupby('uqnr').sum()


In [ ]:
def find_head_maritial_status(data, house_id):
    return 0

In [ ]:
def find_head_education_level(data, house_id):
    return 0